In [1]:
import os
os.environ['ILLUFLY_DOCS'] = 'docs'

# RAG 实现

向量数据库是实现RAG的主要手段，而要使用向量数据库就必须指定文本向量的嵌入模型。

## 朴素的 RAG 实现

ChatAgent 的知识管理可以用来实现简单的 RAG 应用。

In [1]:
from illufly.chat import ChatQwen
from illufly.rag import TextEmbeddings, FaissDB

db = FaissDB(embeddings=TextEmbeddings())
db.add("如果有人问起名字，除非特别指名在问模型的名字，否则应当是指在这个对话场景，此时的名字应当是「illufly」")
db.add("illufly的意思是 illution butter fly 的缩写，是中国古代庄子梦中变蝴蝶的故事")

qwen = ChatQwen(vectordbs=[db])
qwen("一句话回答，illufly是什么？", verbose=True)
qwen.memory

[HUMAN] 一句话回答，illufly是什么？
[AGENT] ChatQwen.4407035888
[INFO] 记住 10 轮对话
[RAG] [{"text": "illufly的意思是 illution butter fly 的缩写，是中国古代庄子梦中变蝴蝶的故事", "meta": {"id": "20241212-72125-0001-0516", "distance": 0.5243071913719177, "source": "", "raw_meta": "", "tags": [], "summary": "illufly的意思是 illution butter fly 的缩写，是中国古代庄子梦中变蝴蝶的故事"}}, {"text": "如果有人问起名字，除非特别指名在问模型的名字，否则应当是指在这个对话场景，此时的名字应当是「illufly」", "meta": {"id": "20241212-72125-0000-0875", "distance": 0.59690922498703, "source": "", "raw_meta": "", "tags": [], "summary": "如果有人问起名字，除非特别指名在问模型的名字，否则应当是指在这个对话场景，此时的名字应当是「illufly」"}}]
illufly是源自庄子梦中变成蝴蝶的故事，意为“illution butterfly”的缩写。
[USAGE] {"input_tokens": 92, "output_tokens": 23, "total_tokens": 115}


[{'role': 'user',
  'content': '回答时请参考已有知识：\n@knowledge\nillufly的意思是 illution butter fly 的缩写，是中国古代庄子梦中变蝴蝶的故事\n\n如果有人问起名字，除非特别指名在问模型的名字，否则应当是指在这个对话场景，此时的名字应当是「illufly」\n\n\n'},
 {'role': 'assistant', 'content': 'ok'},
 {'role': 'user', 'content': '一句话回答，illufly是什么？'},
 {'role': 'assistant',
  'content': 'illufly是源自庄子梦中变成蝴蝶的故事，意为“illution butterfly”的缩写。'}]

**请注意：**

上面的对话记忆显示，自动插入了找到的资料。<br>
实际上，如果是 knowledge 属性中的文本，会被当作不言而喻的常识，全部插入到记忆中。

In [8]:
qwen("你的名字是什么？", verbose=True)
qwen.memory

[INFO] 记住 10 轮对话
我的名字是「illufly」。
[USAGE] {"input_tokens": 128, "output_tokens": 9, "total_tokens": 137}


[{'role': 'user',
  'content': '回答时你必须参考已有信息：\nillufly的意思是 illution butter fly 的缩写，是中国古代庄子梦中变蝴蝶的故事\n如果有人问起名字，除非特别指名在问模型的名字，否则应当是指在这个对话场景，此时的名字应当是「illufly」'},
 {'role': 'assistant', 'content': 'ok'},
 {'role': 'user', 'content': '一句话回答，illufly是什么？'},
 {'role': 'assistant',
  'content': 'illufly是源自中国古代庄子梦中变蝴蝶故事的缩写，代表了「illution butterfly」。'},
 {'role': 'user', 'content': '你的名字是什么？'},
 {'role': 'assistant', 'content': '我的名字是「illufly」。'}]

## 借助 VectorDB 实现 RAG

默认情况下，knowledge 中加载的向量数据库应当加载 `__DOC__` 目录中所有的 markdown 文件。<br>
加载时按照最大 1024K 来切片；在用户提问时，根据问题匹配相似的5个文档片段。

In [9]:
from illufly.embeddings import TextEmbeddings
from illufly.rag import FaissDB
from illufly.chat import ChatQwen

qwen = ChatQwen(knowledge=FaissDB(embeddings=TextEmbeddings()))
qwen("一句话回答，illufly是什么？", verbose=True)
qwen.memory

[INFO] 记住 10 轮对话
illufly 是一个遵循开箱即用、减少新概念、原厂优先及围绕智能体对象等设计理念的AI开发框架，专注于简化AI应用的构建与使用。
[USAGE] {"input_tokens": 2544, "output_tokens": 39, "total_tokens": 2583}


[{'role': 'user',
  'content': "回答时你必须参考已有信息：\n# illufly 设计理念介绍\n\n- [开箱即用的原则](#开箱即用的原则)\n- [减少新概念的原则](#减少新概念的原则)\n- [原厂优先的原则](#原厂优先的原则)\n- [围绕智能体对象的原则](#围绕智能体对象的原则)\n\n## 开箱即用的原则\n\n体验的影响，应用社区构建和AI应用和使用AI\n与 python 自身的风格类似，`illufly` 中也有很多约定替代配置\n\n## 减少新概念的原则\n\nAI 应用场景的一个优势在于，它可以用自然语言交互替代复杂的指令规则，从而大大减少了新概念的学习负担。如果一个开发框架中出现了太多需要记忆的内容，用户的耐心会逐渐消磨殆尽。\n\nillufly 的目标是：在提供新功能的同时，尽量减少新概念的学习，并避免强制记忆。\n\n例如，AI中的常见消息格式通常是这样的字典格式:\n\n```python\n\n\n[\n    {\n        'role': 'system',\n        'content': '你是一个AI助手'\n    },\n    {\n        'role': 'user',\n        'content': '你好'\n    },\n    {\n        'role': 'assistant',\n        'role': '有什么可以帮你？',\n    }\n}\n```\n\n一般的开发框架们为了开发者使用，会提供自己的类定义，例如用下面的代码来替代：\n\n```python\n[\n    SystemMessage('你是一个AI助手'),\n    UserMessage('你好'),\n    AIMessage('有什么可以帮你')\n]\n```\n\n然后要求开发者尽量使用已经创建的 XXMessage 类来封装所有关于消息格式的功能。\n到目前为止，这看起来很不错，也完全符合一般的设计原则。\n开发者一般不会计较这么简洁清晰的消息类定义，而且只有几个从名称看就不言而喻的类，也容易记住。\n\n但问题会逐渐显现。\n\n首先是类定义的体系，仅仅上面几个类是不够的。例如：\n\n- 你一定需要基类，比如：`BaseM

## 借助复杂的 Retriever 实现 RAG

Retriever 可以构建结构复杂的检索器，包括：
- 基于问题扩散的意图理解
- 混合数据搜索
- 检索结果排序

In [2]:
from illufly.chat import ChatQwen
from illufly.rag import Retriever, FaissDB, DashScopeReranker
from illufly.embeddings import TextEmbeddings

retriver = Retriever(
    translators=ChatQwen(),
    searchers=FaissDB(embeddings=TextEmbeddings()),
    reranker=DashScopeReranker()
)

kg = FaissDB(embeddings=TextEmbeddings(), name="kg")

qwen = ChatQwen(knowledge=[kg, retriver], faq=["illufly是一个具有自我进化能力的智能体开发框架"])
qwen("一句话总结illufly是啥？", verbose=True)

[INFO] 记住 10 轮对话
```markdown
<!-- @meta -->
一句话概括illufly的核心功能是什么？
<!-- @meta -->
illufly的主要用途可以用哪句话来描述？
<!-- @meta -->
简述illufly的主要特点和作用？
```
[AGENT] 由 FaissDB.4675646512 检索问题：简述illufly的主要特点和作用？
[AGENT] 由 DashScopeReranker.4675645744 重新排序检索结果

[AGENT] 由 FaissDB.4675646512 检索问题：一句话概括illufly的核心功能是什么？
[AGENT] 由 DashScopeReranker.4675645744 重新排序检索结果

[AGENT] 由 FaissDB.4675646512 检索问题：一句话总结illufly是啥？
[AGENT] 由 DashScopeReranker.4675645744 重新排序检索结果

[AGENT] 由 FaissDB.4675646512 检索问题：illufly的主要用途可以用哪句话来描述？
[AGENT] 由 DashScopeReranker.4675645744 重新排序检索结果

illufly是一个旨在降低AI应用开发门槛、强调开箱即用和减少新概念学习负担的智能体开发框架。
[USAGE] {"input_tokens": 5058, "output_tokens": 28, "total_tokens": 5086}


'illufly是一个旨在降低AI应用开发门槛、强调开箱即用和减少新概念学习负担的智能体开发框架。'